In [9]:
import os
import sys
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image
import imutils


In [10]:
sys.path.append('..')
from task_2a import detect_ArUco_details, mark_ArUco_image

In [11]:

img = cv2.imread('test2.jpg')

img = imutils.resize(img, width=960)

org2 = copy.deepcopy(img)

# ArUco_details_dict, ArUco_corners = detect_ArUco_details(img)
# img = mark_ArUco_image(img, ArUco_details_dict, ArUco_corners) 


# cv2.imshow("Marked Image",img)
cv2.imshow("Marked Image2",img)

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [12]:
print(len(ArUco_corners))

NameError: name 'ArUco_corners' is not defined

Bottom Left Event : 2 C(x-75), 2 C(y+ 75)
Top Left Event : 22 C(y+20), 15 C(y-20)
Mid Left Event : 5 C(x-100), 5 C(y+60)
Top Right Event : 54 C(x+50), 39 C(x-50)
Bottom Right Event : 53 C, 0 C(y+60)

In [ ]:
# print(ArUco_corners[55])
marking_img = np.copy(org2)
centers = copy.deepcopy(ArUco_details_dict)
corners = copy.deepcopy(ArUco_corners)
events = [
    [[corners[7][1][0], corners[21][0][1]], [corners[21][0][0], corners[7][1][1]-10]],
    [corners[28][1], corners[14][0]],
    [corners[31][1], corners[11][3]], 
    [[corners[25][0][0], corners[34][0][1]], [corners[34][0][0], corners[25][0][1]]],    
    [corners[54][2], corners[40][0]]   
]

i=0
eventlist=[]

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("model.tf")
model.eval()
# image = np.expand_dims(image_resized, axis=0)
image_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224), antialias=False),
        
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

for tl, br in events:
    tl_adj = [tl[0] + 10, tl[1] + 7]
    br_adj = [br[0] - 10, br[1] - 4]
    roi = org2[tl_adj[1]:br_adj[1], tl_adj[0]:br_adj[0]]
    i+=1

    # Apply a blur to the image
    blurred = cv2.blur(roi, (5, 5))
    # Apply a bilateral filter to the image
    filtered = cv2.bilateralFilter(roi, d=9, sigmaColor=75, sigmaSpace=75)
    # Perform morphological opening
    kernel = np.ones((5,5),np.uint8)
    opened = cv2.morphologyEx(roi, cv2.MORPH_OPEN, kernel)

    gray = cv2.cvtColor(opened, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to the image
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours in the image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area in descending order, take the first one (the largest)
    contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Get the bounding rectangle of the largest contour
    x, y, w, h = cv2.boundingRect(contour)

    # Crop the image using the bounding rectangle, add some padding if needed
    padding = 0  # adjust this value according to your needs
    crop = roi[max(0, y-padding):min(y+h+padding, roi.shape[0]), max(0, x-padding):min(x+w+padding, roi.shape[1])]
    

    offset_x = tl_adj[0] + x
    offset_y = tl_adj[1] + y    


    eventlist.append(crop)
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "EDSR_x4.pb"
    
    sr.readModel(path)
    
    sr.setModel("edsr",4)
    
    result = sr.upsample(crop)
    with torch.inference_mode():
        # 6. Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = image_transform(result).unsqueeze(dim=0)
        # 7. Make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(transformed_image.to(device))

    # 8. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # 9. Convert prediction probabilities -> prediction labels
    pred = torch.argmax(target_image_pred_probs, dim=1)

    # pred = model.predict(image)
    class_names = ['combat', 'destroyedbuilding', 'fire', 'humanitarianaid', 'militaryvehicles']
    event = class_names[pred]


    offset_x -= 10
    offset_y -= 10
    box = cv2.rectangle(marking_img, (offset_x, offset_y), (offset_x + w + 20, offset_y + h + 20), (0, 255, 0), 2)
    
    offset_y -= 10
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 1
    thickness = 2
    text = event
    (text_width, text_height), _ = cv2.getTextSize(text, font, scale, thickness)

    cv2.rectangle(marking_img, (offset_x, offset_y - text_height - 10), (offset_x + text_width, offset_y), (140, 133, 133), -1)
    cv2.putText(box, event, (offset_x, offset_y - 10), cv2.FONT_HERSHEY_SIMPLEX, scale, (0,255,0), thickness)

cv2.imshow("Original Image", marking_img)

cv2.waitKey(0)
cv2.destroyAllWindows()
